## Module 7 - Network Practice

In this notebook, we will see how to create network visualizations using `networkD3`. 


You may have to **rerun** cells to get the network output for networkD3 examples. 


Here are some references:

- [networkD3 reference](https://cran.r-project.org/web/packages/networkD3/networkD3.pdf)

- [network data sources](http://www-personal.umich.edu/~mejn/netdata/)


## networkD3 Library 

`networkD3` library is an R interface to the **D3 Javascript library**. 


Let's do similar layouts as in the igraph practice notebook for networkD3. **This library produces interactive graphs, you can click on a node and drag it, for example.** 

We need two sets of vertices as input to create a graph from them: source vertices, and target vertices. 

In [ ]:
library(networkD3)

# Create edge data
src <- c("A", "A", "A", "A",
        "B", "B", "C", "C", "D")

target <- c("B", "C", "D", "J",
            "E", "F", "G", "H", "I")

networkData <- data.frame(src, target)

# Plot - may need to run twice 
simpleNetwork(networkData, height=300)

**We can also convert the `igraph` graphs to `D3` graphs.**


In [ ]:
# Let's recreate the gSA graph from the igraph practice notebook here: 

adj_matrix <- as.matrix(read.table("/dsa/data/all_datasets/networks/SAcountries.txt",header=FALSE, sep=" "))
SAnames <- read.csv("/dsa/data/all_datasets/networks/SAcountrylist.txt", header=FALSE)
gSA <-  graph_from_adjacency_matrix(adj_matrix, weighted = NULL,  mode = "undirected")
gSA <- simplify(gSA)
gSA <- set_vertex_attr(gSA, "label", value= as.vector(SAnames$V1))

In [ ]:
gSA_d3 <- igraph_to_networkD3(gSA)


In [ ]:
# Create force directed network plot - you can zoom into this graph with mouse 

forceNetwork(Links = gSA_d3$links, Nodes = gSA_d3$nodes, Source = 'source', Target = 'target', 
             NodeID = 'name', Group = 'name', height=500, zoom=TRUE, fontSize=20)

### IS THIS GRAPH PLANAR ? MOVE THE VERTICES WITH MOUSE TO GET RID OF ALL EDGE CROSSINGS IF YOU CAN.


---




Let's look at a more complex data set. This dataset contains the **co-occurance network of characters in Victor Hugo's novel *Les Misérables*.** 


**A vertex represents a character**, and an **edge between two vertices shows that these two characters appeared in the same chapter** of the the book. The **weight** of each edge indicates how often such a co-appearance occured.

In [ ]:
# This is the Les Miserables data set that comes with the library 
data(MisLinks)
data(MisNodes)

Edges (links) have **weights** that are represented by the `value` column in the data frame as below: 

In [ ]:
head(MisLinks) # EDGE DATA - has weights 

Vertices (nodes) have two attributes: `group` and `size`. 

In [ ]:
head(MisNodes) # VERTEX DATA - has size and grouping 

**Can you identify how many different visual channels represent attributes about the data set below?** 

In [ ]:
# Plot - Move mouse over VERTICES to see character names 

forceNetwork(Links = MisLinks, Nodes = MisNodes,
            Source = "source", Target = "target",
            Value = "value", NodeID = "name",
            Group = "group", opacity = 0.8, height=600, zoom=TRUE, fontSize=30)

**Look at [here](https://www.rdocumentation.org/packages/networkD3/versions/0.4/topics/forceNetwork) for all the options you can use with `forceNetwork()` function.**

### Here are some links that show alternative visualizations for the same data set: 
 - https://bost.ocks.org/mike/miserables/
 - https://studentwork.prattsi.org/infovis/labs/character-networks-visualization-for-les-miserables/
 - https://studentwork.prattsi.org/infovis/visualization/les-miserables-character-network-visualization/

### YOUR TURN: 

**Create an interactive network** for the South America data from above. You'll need to create two data frames similar to the `Mis` example. 

In [ ]:
< YOUR CODE HERE >

Now, **find neighbors of Bolivia** using the `neighbors()` function. For that, you'll need an `igraph` graph. 

In [ ]:
< YOUR CODE HERE >


---


**We can also read graphs in GML format.** 

This example contains an undirected social network of frequent associations between 62 dolphins in a community living off Doubtful Sound, New Zealand, as compiled by Lusseau et al. (2003). 

In [ ]:
gml_data <- read.graph("/dsa/data/all_datasets/networks/dolphins.gml", format=c("gml"))
head(gml_data)

In [ ]:
gml_data <- simplify(gml_data)

dol <- igraph_to_networkD3(gml_data)

forceNetwork(Links = dol$links, Nodes = dol$nodes, NodeID = "name", Group = "name", height=600, zoom=TRUE)

**With this data set, we can experiment with community detection functions of `igraph` by clustering vertices like this:** 

In [ ]:
cfg <- cluster_fast_greedy(as.undirected(gml_data))
plot(cfg, gml_data)

In [ ]:
ceb <- cluster_edge_betweenness(gml_data)
plot(ceb, gml_data)

In [ ]:
clp <- cluster_label_prop(gml_data)
plot(clp, gml_data)

### Radial Networks

We can also read network data in JSON format like in the example below: 

In [ ]:
# example of a radial network given in Javascript's JSON format.
library(jsonlite)
URL <- "https://raw.githubusercontent.com/christophergandrud/networkD3/master/JSONdata/flare.json"

## Convert the data to list format
Flare <- fromJSON(URL, simplifyDataFrame = FALSE)

In [ ]:
# JSON format
Flare

In [ ]:
# Use subset of data for a more readable diagram
Flare$children = Flare$children[1:3]

radialNetwork(List = Flare, fontSize = 20, opacity = 1, height=600)

We can **visualize clusters** with a radial network like this: 

In [ ]:
hc <- hclust(dist(USArrests), "ave")
radialNetwork(as.radialNetwork(hc))